In [2]:
import pandas as pd
import numpy as np
import gc
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [1]:
import os
import sys

os.environ['TF_FORCE_GPU_ALLOW_GROWTH']='true'

os.environ['CUDA_VISIBLE_DEVICES'] = '5'


### LOAD DFs

In [3]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if( p in actual) and (p not in predicted[:i]):
            #print(p)
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

#### Что нужно?

    Для работы нужны два файла с новостями под историю 5  news_for_history5.csv и нарезка юзер-стори под историю 5 narezka_last5_history_users.csv

In [4]:
#тут лежат все новости в нужном формате
df_news = pd.read_csv('news_for_history5.csv', sep='^')
#тут лежат только просмотры каждым юзером + предыстория в 5 последних тегов - поэтому сделан шифт = 5 
user5story = pd.read_csv('narezka_last5_history_users.csv', sep='^') 

In [5]:
df_news.head()

,text_prev,id,text_title,news_tags,spheeres,images_cnt,news_url,news_date,full_text,full_tags
0,Электронное голосование доступно для граждан Р...,75178073,Открыта запись на электронное голосование по и...,"просмотренные теги:конституция, поправки, элек...",просмотренные сферы:социальная сфера,18,mos.ru/news/item/75178073/,2020-06-04 22:14:43,открыта запись на электронное голосование по и...,"просмотренные теги:конституция, поправки, элек..."
1,Они будут следовать от станции метро «Молодежн...,80375073,Для пассажиров закрытого участка Арбатско-Покр...,"просмотренные теги:метро, ограничения, компенс...","просмотренные сферы:транспорт, мой район, комф...",18,mos.ru/news/item/80375073/,2020-09-25 16:49:20,для пассажиров закрытого участка арбатско-покр...,"просмотренные теги:метро, ограничения, компенс..."
2,"Арина Аверина победила с результатом 79,250 ба...",41116073,Москвичка Арина Аверина выиграла чемпионат Евр...,"просмотренные теги:соревнования, художественна...",просмотренные сферы:спорт,18,mos.ru/news/item/41116073/,2018-06-04 09:51:15,москвичка арина аверина выиграла чемпионат евр...,"просмотренные теги:соревнования, художественна..."
3,Новое здание появится на Волгоградском проспекте.,94978073,Многофункциональный комплекс со спортивными и ...,"просмотренные теги:сергей лёвкин, многофункцио...",просмотренные сферы:строительство и реконструкция,18,mos.ru/news/item/94978073/,2021-08-23 09:07:27,многофункциональный комплекс со спортивными и ...,"просмотренные теги:сергей лёвкин, многофункцио..."
4,"Фаина Рублева — о новейших технологиях, подаре...",64742073,«По масштабам Вселенной 90 лет — это миг». Инт...,"просмотренные теги:интервью, юбилеи, истории, ...","просмотренные сферы:культура, музейные истории...",18,mos.ru/news/item/64742073/,2019-11-01 21:28:46,«по масштабам вселенной 90 лет — это миг». инт...,"просмотренные теги:интервью, юбилеи, истории, ..."


In [6]:
user5story.head()

,text_query,text_trgt,text_env,text_trgt_env,rank,uid,id,id_history,date_time,news_date
0,дома-порталы: где в москве можно почувствовать...,московские школьники завоевали четыре медали н...,"просмотренные теги:архитектура, особняки, дохо...","просмотренные теги:экономика, награды, школьни...",1,1,94098073.0,94006073.0|94000073.0|94062073.0|94063073.0|93...,2021-08-04 09:15:14,2021-08-02 12:21:23
1,чтобы дома было комфортно: какие вопросы помог...,две новостройки для переселения по программе р...,"просмотренные теги:перепланировка, жку, дома, ...","просмотренные теги:реновация, программа ренова...",1,1,94106073.0,94000073.0|94062073.0|94063073.0|93893073.0|94...,2021-08-04 09:15:38,2021-08-02 15:45:42
2,полярные совы в московском зоопарке обзавелись...,чат-бот поможет врачам больше узнать о здоровь...,"просмотренные теги:культура, птицы, московский...","просмотренные теги:здравоохранение, технологии...",1,1,94108073.0,94062073.0|94063073.0|93893073.0|94098073.0|94...,2021-08-04 16:55:38,2021-08-02 16:13:31
3,в москве начали производить лифты с системой б...,морские звезды и танцы нерп: как отметят шести...,"просмотренные теги:технологии, лифты, владимир...","просмотренные теги:вднх, празднования, москвар...",1,1,94132073.0,94063073.0|93893073.0|94098073.0|94106073.0|94...,2021-08-04 16:55:45,2021-08-02 20:45:00
4,любовь и звери: как разводят редких животных в...,"карты, лыжи, два весла: как туризм в ссср стал...","просмотренные теги:зоопарк, леопарды, амурские...","просмотренные теги:туризм, документы, туристы,...",1,1,94057073.0,93893073.0|94098073.0|94106073.0|94108073.0|94...,2021-08-04 16:56:02,2021-07-30 17:45:09


In [7]:
# ВАЖНО: каждый юзер внутри своего uid отсортирован по дате от прошлого к будущему
user5story[user5story['uid']==1].head()

,text_query,text_trgt,text_env,text_trgt_env,rank,uid,id,id_history,date_time,news_date
0,дома-порталы: где в москве можно почувствовать...,московские школьники завоевали четыре медали н...,"просмотренные теги:архитектура, особняки, дохо...","просмотренные теги:экономика, награды, школьни...",1,1,94098073.0,94006073.0|94000073.0|94062073.0|94063073.0|93...,2021-08-04 09:15:14,2021-08-02 12:21:23
1,чтобы дома было комфортно: какие вопросы помог...,две новостройки для переселения по программе р...,"просмотренные теги:перепланировка, жку, дома, ...","просмотренные теги:реновация, программа ренова...",1,1,94106073.0,94000073.0|94062073.0|94063073.0|93893073.0|94...,2021-08-04 09:15:38,2021-08-02 15:45:42
2,полярные совы в московском зоопарке обзавелись...,чат-бот поможет врачам больше узнать о здоровь...,"просмотренные теги:культура, птицы, московский...","просмотренные теги:здравоохранение, технологии...",1,1,94108073.0,94062073.0|94063073.0|93893073.0|94098073.0|94...,2021-08-04 16:55:38,2021-08-02 16:13:31
3,в москве начали производить лифты с системой б...,морские звезды и танцы нерп: как отметят шести...,"просмотренные теги:технологии, лифты, владимир...","просмотренные теги:вднх, празднования, москвар...",1,1,94132073.0,94063073.0|93893073.0|94098073.0|94106073.0|94...,2021-08-04 16:55:45,2021-08-02 20:45:00
4,любовь и звери: как разводят редких животных в...,"карты, лыжи, два весла: как туризм в ссср стал...","просмотренные теги:зоопарк, леопарды, амурские...","просмотренные теги:туризм, документы, туристы,...",1,1,94057073.0,93893073.0|94098073.0|94106073.0|94108073.0|94...,2021-08-04 16:56:02,2021-07-30 17:45:09


в обоих файла текстовые поля формировались так как ниже - к тегам добавляли префикс 'просмотренные теги:' к сферам 'просмотренные сферы:'
далее это описание складывалось с разделителем точка

    df_news['full_text']=df_news['text_title'].str.lower()#+', '+df_news['text_prev'].str.lower()
    df_news['news_tags']=df_news['news_tags'].map(lambda x: 'просмотренные теги:'+', '.join(x).lower())
    df_news['spheeres']=df_news['spheeres'].map(lambda x: 'просмотренные сферы:'+', '.join(x).lower())
    df_news['full_tags']=df_news['news_tags']+'. '+df_news['spheeres']



In [8]:
#далее получим теги для новостей кандидатов которые положим в FAISS

In [9]:
cands_tags = df_news['full_tags']
tags_news_ids = df_news['id'].values

In [10]:
#далее покажем как извлекать юзер стори по тегам

In [11]:
#тк в предыстории у нас трейн сет: последние 5 собыйти + 1 на предсказание, 
#то для инференса надо сложить их и взять 5 последних включая "предсказание"
#тк на момент теста это уже произошло

user5story['text_env'] = user5story['text_env'].map(lambda x: '|'.join( x.split('|')[1:] ) )+'|'+user5story['text_trgt_env']
user5story['text_query'] = user5story['text_query'].map(lambda x: '|'.join( x.split('|')[1:] ) )+'|'+user5story['text_trgt']

In [12]:
user_5_tags = user5story['text_env'].str.split('|').values
user_5_ids = user5story['uid'].values

In [13]:
#как видите тут лежат 5 последних просмотров на определенную дату для юзера 1
user_5_tags[0], user_5_ids[0], user5story['date_time'][0]

(['просмотренные теги:перепланировка, жку, дома, лицензии, мосжилинспекция, олег кичиков. просмотренные сферы:городское хозяйство',
  'просмотренные теги:культура, птицы, московский зоопарк, белые совы. просмотренные сферы:культура, мой район, животные, комфорт: досуговая инфраструктура',
  'просмотренные теги:технологии, лифты, владимир ефимов, щербинский лифтостроительный завод, александр прохоров, умные лифты. просмотренные сферы:технологии',
  'просмотренные теги:зоопарк, леопарды, амурские тигры, рептилии, центр воспроизводства редких видов животных, дрофа. просмотренные сферы:животные',
  'просмотренные теги:экономика, награды, школьники, олимпиады. просмотренные сферы:образование'],
 1,
 '2021-08-04 09:15:14')

In [14]:
#тк нам нужно сделать предсказание для всех юзеров относительно их последних 5 действий в базе 
#делаем так
last_story_uid = {uid: user5story['text_env'][user5story.uid==uid][-1:].str.split('|').values for uid in user5story['uid'].unique()}

In [15]:
#проверим так ли это
user5story['text_env'][user5story.uid==10].values[-1]

'просмотренные теги:детские сады, москомархитектура, юлиана княжевская. просмотренные сферы:строительство и реконструкция, образование|просмотренные теги:реконструкция, строительство, школы, зил, жилая недвижимость, тюфелева роща. просмотренные сферы:образование, комфорт, планы|просмотренные теги:благоустройство, здравоохранение, поликлиники, сергей собянин, бибирево, мой район. просмотренные сферы:здравоохранение, мой район, комфорт, мой район: избранное, город: детали|просмотренные теги:благоустройство, сергей собянин, парк горького, пешеходные маршруты. просмотренные сферы:парки и пешеходные зоны|просмотренные теги:парк горького, нескучный сад, история москвы, зелёный театр, девушка с веслом. просмотренные сферы:культура, музейные истории, уникальность, уникальность: знаковые культурно-исторические объекты'

In [16]:
last_story_uid[10]

array([list(['просмотренные теги:детские сады, москомархитектура, юлиана княжевская. просмотренные сферы:строительство и реконструкция, образование', 'просмотренные теги:реконструкция, строительство, школы, зил, жилая недвижимость, тюфелева роща. просмотренные сферы:образование, комфорт, планы', 'просмотренные теги:благоустройство, здравоохранение, поликлиники, сергей собянин, бибирево, мой район. просмотренные сферы:здравоохранение, мой район, комфорт, мой район: избранное, город: детали', 'просмотренные теги:благоустройство, сергей собянин, парк горького, пешеходные маршруты. просмотренные сферы:парки и пешеходные зоны', 'просмотренные теги:парк горького, нескучный сад, история москвы, зелёный театр, девушка с веслом. просмотренные сферы:культура, музейные истории, уникальность, уникальность: знаковые культурно-исторические объекты'])],
      dtype=object)

### LOAD FREEZE_GRAPH FOR INF

Когда последние данные получены можно перейти к примеру загрузки и использования модели

In [17]:
import logging
import numpy as np
import tensorflow as tf

from tensorflow.python.estimator.estimator import Estimator
from tensorflow.python.estimator.run_config import RunConfig
from tensorflow.python.estimator.model_fn import EstimatorSpec
from tensorflow.keras.utils import Progbar
from text_preprocessing import FullTokenizer

from tensorflow.python.client import device_lib

def build_proto_config(cpu_cores=4, allow_growth=True, use_gpu=True):
    config = tf.ConfigProto(device_count={'GPU': 1 if use_gpu else 0})
    config.gpu_options.allow_growth = allow_growth
    return config


def get_configurable_session_and_graph(config=None):
    if config is None:
        config = build_proto_config()

    graph = tf.Graph()
    sess = tf.Session(config=config, graph=graph)

    return {
        "session": sess,
        "graph": graph
    }

def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]
        
        
def _get_masks(tokens, max_seq_length):
    if len(tokens)>max_seq_length:
        raise IndexError("Token lengath more than max seq length!")
    return [1]*len(tokens)+[0]*(max_seq_length-len(tokens))

def _get_segments(tokens, max_seq_length):
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [0]*len(tokens)+[0]*(max_seq_length-len(tokens))

def _trim_input(text, max_seq_length):

    t = tokenizer.tokenize(text)
        
    if len(t) > max_seq_length - 2:
        t = t[0 : (max_seq_length - 2)]
        
    return t

def _get_ids(tokens, tokenizer, max_seq_length):
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0]*(max_seq_length - len(token_ids))
    return input_ids

def _convert_to_bert_inputs(text, tokenizer, max_seq_length):
    #stacked token
    stoken = ['[CLS]'] + text + ['[SEP]'] 
    input_ids = _get_ids(tokens=stoken, tokenizer=tokenizer,max_seq_length=max_seq_length)
    input_masks = _get_masks(tokens=stoken, max_seq_length=max_seq_length)
    input_segments = _get_segments(tokens=stoken, max_seq_length=max_seq_length)
    
    return [input_ids, input_masks, input_segments]

def compute_input_array(texts, tokenizer, max_seq_length):
    input_ids, input_masks, input_segments = [], [], []
    for text in texts:
        t = text
        t  = _trim_input(t, max_seq_length)
        ids, masks, segments = _convert_to_bert_inputs(t, tokenizer, max_seq_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return [np.array(input_ids, dtype=np.int32), np.array(input_masks, dtype=np.int32), 
            np.array(input_segments, dtype=np.int32)]

class SBERT():
    def __init__(self, model_path, tokenizer, seq_len=24):
        self.max_seq_length=seq_len
        self.tokenizer = tokenizer
        self.restored_graph=self._load_graph(model_path)
        self.sess = tf.Session(graph=self.restored_graph)
        #первый запуск всегда долгий поэтому в конструкторе
        self.x_id, self.x_mask, self.x_seg, self.y1 = self.restore_model(self.restored_graph)
        _ = self.predict([[0]*self.max_seq_length], [[0]*self.max_seq_length], [[0]*self.max_seq_length])
        
        
    def _load_graph(self, frozen_graph_filename):
        with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())

        with tf.Graph().as_default() as graph:
            tf.import_graph_def(graph_def)
        return graph

    def restore_model(self, restored_graph):

        graph_ops = restored_graph.get_operations()
        input_op, output_op = [graph_ops[0].name, graph_ops[1].name, graph_ops[2].name], [graph_ops[-1].name]
        
        print(input_op, output_op)
        
        [x_id, x_mask, x_seg] = [restored_graph.get_tensor_by_name(input_op[i] + ':0') for i,k in enumerate(input_op)]
        y1 = restored_graph.get_tensor_by_name(output_op[0] + ':0')
        return x_id, x_mask, x_seg, y1
    
    def predict(self, in_ids, in_masks, in_segs):
        return self.sess.run( self.y1, 
                             feed_dict={self.x_id: in_ids, self.x_mask: in_masks, self.x_seg: in_segs})
    

    def get_embs(self, texts):
        
        if type(texts)!=list:
            texts=[texts]
            
        in_ids, in_masks, in_segs = compute_input_array(texts, self.tokenizer, self.max_seq_length)
        
        outs = self.predict(in_ids, in_masks, in_segs)
        return {'sentence_embs': outs}

    
def build_scu_model(ctx_len = 5, dim=768):
    
    inp_emb = tf.keras.layers.Input(shape=(ctx_len, dim), dtype=tf.float32, name="nlu_input")
    
    d1 = tf.keras.layers.Dense(1024, activation='tanh', name='d1')
    d2 = tf.keras.layers.Dense(768, activation='tanh',  name='d2')
    
    enc_q = tf.reshape(inp_emb, (-1, ctx_len, 768))
    enc_q = tf.keras.layers.GlobalAveragePooling1D()(enc_q)
    enc_q = d1(enc_q)
    enc_q = d2(enc_q)

    final_state_encoded = tf.identity(enc_q, "state_output")
    t_model = tf.keras.models.Model(inputs=[inp_emb], outputs=[final_state_encoded])
    
    return t_model

In [18]:
vocab_path = 'vocab.txt'

max_sequence_length = 24
tokenizer = FullTokenizer(vocab_file=vocab_path, do_lower_case=True)

bes = SBERT(model_path='rene_labse.pbtxt',
            tokenizer=tokenizer,
            seq_len=24)

rene_mlp = build_scu_model()
rene_mlp.load_weights('rene_mlp.h5')

['import/tok_input', 'import/msk_input', 'import/seg_input'] ['import/pooled_output']


In [19]:
#возьмем последние 5 запросов для usera 1
request = last_story_uid[1]

In [20]:
request[0]

['просмотренные теги:парки, экотропы, петр бирюков, настилы. просмотренные сферы:городское хозяйство, мой район, комфорт',
 'просмотренные теги:награды, школьники, олимпиады, международная географическая олимпиада. просмотренные сферы:образование',
 'просмотренные теги:отопление, подготовка, отопительный сезон, жилые дома, петр бирюков, системы теплоснабжения. просмотренные сферы:городское хозяйство',
 'просмотренные теги:реконструкция, москомархитектура, юлиана княжевская, нмиц глазных болезней имени гельмгольца. просмотренные сферы:строительство и реконструкция, здравоохранение, комфорт: социальная инфраструктура: здравоохранение',
 'просмотренные теги:экология, электробусы, сергей собянин, контракты жизненного цикла. просмотренные сферы:транспорт']

In [21]:
#чтобы получить контекстный вектор для поиска - необходимо сначала запустить bert_encoder - bes
#далее отправить в контекстную модель менеджера для отображения в контекст rene_mlp
r = bes.get_embs(request[0])['sentence_embs']
v = rene_mlp.predict(np.array([r]))

In [22]:
r.shape, v.shape #r-хранит изначальные вектора контекста, а v-перевзвешенный контекстный вектор относительно рекомендации

((5, 768), (1, 768))

In [23]:
#чтобы получить рекомендацию для юзера 10 по его последнему вектору истории V надо векторизовать базу новостей
#базу новостей векторизуем просто с помощью bes модели (тк она без контекстная)
cands_tags=list(cands_tags.values)

In [24]:
basa_news_vecs= bes.get_embs(cands_tags)['sentence_embs']

In [25]:
basa_news_vecs.shape

(6554, 768)

In [26]:
user5story[user5story.uid == 10][:-1]['id'].values

array([85482073.,  6906050.,  6670050., 88964073., 93886073., 91165073.,
       65016073., 82164073.,  7520050., 80352073., 48324073.,  7529050.,
       90368073., 48401073., 41498073.,  5528050.,  6285050.,  5529050.,
       92805073.,  5397050., 92551073.,  5849050., 78167073.,  6689050.,
       85674073., 92118073.,  7096050.,  7513050., 58865073., 55453073.,
       58932073.,  4872050., 92856073.,  5970050., 93578073.,  7560050.,
       93992073., 88862073., 89421073., 93664073., 87703073.,  7400050.,
        7399050.,  7280050.,  7176050.,  6638050.,  6726050.,  6550050.,
        7073050.,  5825050.,  4847050., 60088073.,  5566050.,  6335050.,
        5038050.])

In [113]:
#найдем топ100 ответов к ремонедации и отфильтруем по предыстории уже просмотренные

In [114]:
scores = v @ basa_news_vecs.T #в ФАИСС просто дот прод (а тут это матрич перемножение поэтому транспонируем)
preds = np.argsort(-1*scores)[:,:100].flatten() #id лучших топ100 в индексе
y_hist = user5story[user5story.uid == 10]['id'].values
top100_cans = [t for k,t in enumerate(list(tags_news_ids[preds])) if t not in y_hist] #переводим id индексав id новости tags_news_ids

In [115]:
top100_cans[:20]

[94636073,
 94722073,
 94688073,
 94679073,
 94696073,
 94669073,
 94707073,
 94703073,
 7573050,
 94605073,
 94699073,
 94720073,
 7574050,
 94702073,
 94701073,
 93588073,
 94736073,
 94794073,
 94717073,
 94534073]

In [116]:
#Теперь посмотрим на последние 5 просмотров и что рекомендует по ним
#Это просмотры
user5story[user5story.uid == 10][-1:]['text_query'].values[0].split('|')

['школа и детский сад: здание-трансформер построят в районе тропарево-никулино',
 'сергей собянин: на территории бывшей промзоны «зил» появится самая большая школа в россии',
 'сергей собянин: в москве реализуется огромная программа модернизации поликлиник',
 'в парке горького открыли вход со стороны ленинского проспекта',
 'почему сад нескучный, театр зеленый, а девушка — с веслом: пять историй из парка горького']

In [117]:
#Его теги
user5story[user5story.uid == 10][-1:]['text_env'].values[0].split('|')

['просмотренные теги:детские сады, москомархитектура, юлиана княжевская. просмотренные сферы:строительство и реконструкция, образование',
 'просмотренные теги:реконструкция, строительство, школы, зил, жилая недвижимость, тюфелева роща. просмотренные сферы:образование, комфорт, планы',
 'просмотренные теги:благоустройство, здравоохранение, поликлиники, сергей собянин, бибирево, мой район. просмотренные сферы:здравоохранение, мой район, комфорт, мой район: избранное, город: детали',
 'просмотренные теги:благоустройство, сергей собянин, парк горького, пешеходные маршруты. просмотренные сферы:парки и пешеходные зоны',
 'просмотренные теги:парк горького, нескучный сад, история москвы, зелёный театр, девушка с веслом. просмотренные сферы:культура, музейные истории, уникальность, уникальность: знаковые культурно-исторические объекты']

In [118]:
#Далее рекомендации
df_news[df_news['id'].isin(top100_cans[:20])]['text_title'].values

array(['Экотропу со смотровыми площадками и качелями откроют в будущем парке «Яуза» в сентябре',
       'В сервисе «Вместе с культурой» появились подборки помещений для разных мероприятий',
       'Модернизация и реставрация: как ремонтировали Большой Каменный мост',
       'В поселке Марьино построят детский сад на 200 мест',
       'Москва поднялась на четыре позиции в рейтинге инновационных городов Европы',
       'Джаз, соул и ритм-н-блюз на ВДНХ: каким будет новый сезон проекта «Музыка на крыше»',
       'Судьба артефакта: как археологические находки попадают в музеи',
       'В пойме реки Чермянки появится два памп-трека',
       'Больше половины жилых домов подготовили к отопительному сезону',
       '92 процента городских объектов подготовили к отопительному сезону',
       'Надежный старт: как начать карьеру в Московском метрополитене',
       'Корпус центра глазных болезней имени Гельмгольца реконструируют',
       'Пятиэтажку в Ростокине расселили по программе реновации',
  

In [119]:
df_news[df_news['id'].isin(top100_cans[:20])]['news_tags'].values

array(['просмотренные теги:парки, экотропы, петр бирюков, настилы',
       'просмотренные теги:культура, дома культуры, мероприятие, вместе с культурой',
       'просмотренные теги:реставрация, мосты, капитальный ремонт, большой каменный мост',
       'просмотренные теги:тинао, дети, детские сады, валерий леонов, филимонковское',
       'просмотренные теги:рейтинги, наталья сергунина, владимир ефимов, александр прохоров, инвестиции инновации',
       'просмотренные теги:вднх, концерты, кино, рабочий и колхозница, саундтреки, музыка на крыше',
       'просмотренные теги:реставрация, история, археология, находки, артефакты',
       'просмотренные теги:благоустройство, спорт, реки, памп-треки, чермянка',
       'просмотренные теги:отопление, подготовка, отопительный сезон, жилые дома, петр бирюков',
       'просмотренные теги:отопление, подготовка, отопительный сезон, жилые дома, петр бирюков, системы теплоснабжения',
       'просмотренные теги:трудоустройство, московский метрополитен, мо

In [120]:
df_news[df_news['id'].isin(top100_cans[:20])]['spheeres'].values

array(['просмотренные сферы:городское хозяйство, мой район, комфорт',
       'просмотренные сферы:культура, имущественно-земельные отношения, технологии',
       'просмотренные сферы:строительство и реконструкция, городское хозяйство, мой район, уникальность: знаковые культурно-исторические объекты, качество: благоустройство, мой район: избранное',
       'просмотренные сферы:строительство и реконструкция, мой район, комфорт: социальная инфраструктура: образование',
       'просмотренные сферы:экономика и предпринимательство, достижения',
       'просмотренные сферы:парки и пешеходные зоны',
       'просмотренные сферы:археология и памятники архитектуры, музейные истории',
       'просмотренные сферы:парки и пешеходные зоны, мой район, благоустройство',
       'просмотренные сферы:городское хозяйство',
       'просмотренные сферы:городское хозяйство',
       'просмотренные сферы:транспорт',
       'просмотренные сферы:строительство и реконструкция, здравоохранение, комфорт: социальная 

### Выводы:
    Как видим модель понимает из языка интересы о детсадах и школах а так же благоустройстве и рекомендует все вокруг этой тематики
    и даже учитывает возможность для разнообразия предложить новости инноваций для благоустройства города

In [51]:
df_news.iloc[5956]

text_prev     Новая станция, которая входит в состав Третьег...
id                                                      3794050
text_title    Строительство станции метро «Нижняя Масловка» ...
news_tags         просмотренные теги:метро, тпк, сергей собянин
spheeres      просмотренные сферы:строительство и реконструкция
images_cnt                                                   18
news_url                      mos.ru/mayor/themes/4299/3794050/
news_date                                   2017-01-23 14:47:52
full_text     строительство станции метро «нижняя масловка» ...
full_tags     просмотренные теги:метро, тпк, сергей собянин....
Name: 5956, dtype: object

In [ ]:
preds_all = []
for id_ in last_story_uid.keys():
    print('prepare recs for uid', id_)
    request = last_story_uid[id_]
    r = bes.get_embs(request[0])['sentence_embs']
    v = rene_mlp.predict(np.array([r]))
    scores = v @ basa_news_vecs.T #в ФАИСС просто дот прод (а тут это матрич перемножение поэтому транспонируем)
    preds = np.argsort(-1*scores)[:,:100].flatten() #id лучших топ100 в индексе
    int_ids = np.argsort(scores)[:,:1].flatten()
    y_hist = user5story[user5story.uid == id_]['id'].values
    top100_cans = [t for k,t in enumerate(list(tags_news_ids[preds])) if t not in y_hist] #переводим id индексав id новости tags_news_ids
    
    preds_all.append(top100_cans[:20])
    
print('--Done--')

prepare recs for uid 1
prepare recs for uid 2
prepare recs for uid 3
prepare recs for uid 4
prepare recs for uid 5
prepare recs for uid 6
prepare recs for uid 7
prepare recs for uid 8
prepare recs for uid 9
prepare recs for uid 10
prepare recs for uid 11
prepare recs for uid 13
prepare recs for uid 14
prepare recs for uid 16
prepare recs for uid 17
prepare recs for uid 18
prepare recs for uid 19
prepare recs for uid 20
prepare recs for uid 21
prepare recs for uid 22
prepare recs for uid 23
prepare recs for uid 24
prepare recs for uid 25
prepare recs for uid 26
prepare recs for uid 27
prepare recs for uid 28
prepare recs for uid 30
prepare recs for uid 31
prepare recs for uid 32
prepare recs for uid 33
prepare recs for uid 34
prepare recs for uid 35
prepare recs for uid 36
prepare recs for uid 37
prepare recs for uid 38
prepare recs for uid 39
prepare recs for uid 41
prepare recs for uid 42
prepare recs for uid 43
prepare recs for uid 44
prepare recs for uid 45
prepare recs for uid 46
p

In [32]:
preds_all=np.vstack(preds_all)
preds_all.shape

(239, 20)

In [33]:
preds_all

array([[94636073, 94722073, 94688073, ..., 89939073, 94802073, 94612073],
       [87880073, 80034073, 83063073, ..., 91791073, 90453073, 89619073],
       [92528073, 76373073, 76398073, ..., 76501073, 68112073,  7069050],
       ...,
       [93614073, 93566073, 93383073, ..., 82777073, 93317073, 93883073],
       [92933073, 93176073, 93636073, ..., 78790073, 77358073, 92700073],
       [88957073, 94179073, 94133073, ..., 94360073, 87290073, 94350073]])

In [36]:
subm = pd.DataFrame({'user_id': list(last_story_uid.keys())})
for i in range(20):
    subm['book_id_'+str(i+1)]=preds_all[:, i]

In [37]:
subm

,user_id,book_id_1,book_id_2,book_id_3,book_id_4,book_id_5,book_id_6,book_id_7,book_id_8,book_id_9,...,book_id_11,book_id_12,book_id_13,book_id_14,book_id_15,book_id_16,book_id_17,book_id_18,book_id_19,book_id_20
0,1,94636073,94722073,94688073,94696073,94707073,94703073,94699073,94720073,94702073,...,94736073,94794073,94717073,94534073,94777073,7576050,94757073,89939073,94802073,94612073
1,2,87880073,80034073,83063073,6778050,62372073,89747073,91103073,88374073,77742073,...,93998073,78505073,70148073,92882073,75418073,91444073,90698073,91791073,90453073,89619073
2,3,92528073,76373073,76398073,7063050,76096073,76581073,76207073,93091073,93633073,...,77193073,79114073,79047073,76377073,78225073,77436073,77521073,76501073,68112073,7069050
3,4,84980073,82624073,84506073,92446073,94660073,56168073,88791073,88859073,52858073,...,77605073,84922073,92216073,93321073,26285073,84953073,87070073,32578073,53214073,84762073
4,5,75262073,91772073,78309073,79677073,95147073,90634073,47732073,86711073,84909073,...,69843073,82354073,5983050,7598050,88395073,91240073,7356050,94845073,91298073,84231073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,274,68814073,31720073,49324073,66495073,64284073,84786073,85312073,84917073,68712073,...,5804050,83095073,67966073,72572073,56991073,55779073,21298073,56743073,44677073,61624073
235,275,85593073,73249073,88635073,47793073,72686073,74127073,83216073,80320073,85591073,...,61147073,65031073,22115073,56991073,56494073,21298073,42041073,80224073,88769073,79191073
236,276,93614073,93566073,93383073,78790073,94153073,78312073,76078073,94853073,93967073,...,93073073,94039073,93767073,81299073,93452073,81771073,93971073,82777073,93317073,93883073
237,277,92933073,93176073,93636073,93031073,93623073,87173073,76078073,87317073,94994073,...,92358073,93475073,81299073,93852073,94853073,81771073,93614073,78790073,77358073,92700073


In [38]:
subm.to_csv('pypypy_case_10_answers.csv', sep=';', index=False)